## Тут описан основной алгоритм получения 3d карты

1. Подключаем все необходимые библиотеки и загружаем данные с калиброки

In [1]:
import cv2
import numpy as np 
import glob
from tqdm import tqdm
import PIL.ExifTags
import PIL.Image
from matplotlib import pyplot as plt 
import math

In [2]:
# and projection matrix in the new (rectified) coordinate systems for the second camera.
line = './calibration/camera_params/stereo_params/'
ret = np.load(line + 'ret.npy')
K_left = np.load(line + 'K_left.npy')
K_right = np.load(line + 'K_right.npy')
dist_left = np.load(line + 'dist_left.npy')
dist_right = np.load(line + 'dist_right.npy')
R = np.load(line + 'R.npy')
T = np.load(line + 'T.npy')
image_size = np.load(line + 'image_size.npy')
image_size = (image_size[1], image_size[0])



2. Учитывая внутренние коэффициенты и коэффициенты искажения для обеих камер, а также перемещение и вращение, которые связаны с расположением одной камеры относительно другой, stereoRectify () рассчитывает для нас выпрямление, проекцию, и матрицу диспаратности, которые нам нужны для извлечения информации о глубине из стереоизображений с этой пары камер.

In [3]:
R_left, R_right, P_left, P_right, Q, roi_left, roi_right = cv2.stereoRectify(cameraMatrix1 = K_left, 
                                                  cameraMatrix2 = K_right, 
                                                  distCoeffs1 = dist_left, 
                                                  distCoeffs2 = dist_right, 
                                                  imageSize = image_size, 
                                                  R = R, 
                                                  T = T,
                                                  alpha=1)

In [4]:
roi_left

(204, 167, 2264, 1604)

In [5]:
roi_right

(129, 169, 2238, 1590)

In [6]:
roi = (204, 169, 2238, 1590)

3. Функция, которая реализует математику, изображенную на рисунке ниже, называется initUndistortRectifyMap(). Мы вызываем эту функцию дважды, один раз для левой и один раз дляправильное изображение стереопары:
<img src="1.png">

In [7]:
map1_left, map2_left = cv2.initUndistortRectifyMap( cameraMatrix = K_left,
                                         distCoeffs = dist_left, 
                                         R = R_left,
                                         newCameraMatrix = P_left, 
                                         size = image_size,
                                         m1type = cv2.CV_32FC2)
map1_right, map2_right = cv2.initUndistortRectifyMap( cameraMatrix = K_right,
                                         distCoeffs = dist_right, 
                                         R = R_right,
                                         newCameraMatrix = P_right, 
                                         size = image_size,
                                         m1type = cv2.CV_32FC2)

4. Подготовительные работы выполнены, сейчас можно брать два изображения и обрабатывать их. На данном шаге загружаем изображения.

In [8]:
#Specify image paths
img_path1 = './calibration/calibration_images/new/25791059/87.png'
img_path2 = './calibration/calibration_images/new/25797059/87.png'



#Load pictures
img_1 = cv2.imread(img_path1)
img_2 = cv2.imread(img_path2)

5. Применяем к ним всё, что было на рисунке

In [9]:
dst_left = cv2.remap(src = img_1,
                     map1 = map1_left,
                     map2 = map2_left,
                     interpolation = cv2.INTER_NEAREST)
dst_right = cv2.remap(src = img_2,
                     map1 = map1_right,
                     map2 = map2_right,
                     interpolation = cv2.INTER_NEAREST)

In [10]:

#plt.imshow(dst_left)
cv2.imwrite('dst_left.png', dst_left)
#plt.imshow(dst_right)
cv2.imwrite('dst_right.png', dst_right)

True

In [11]:
def crop_image(image, border):
    return image[border[1]:border[3], border[0]:border[2]]

In [12]:
dst_left_crop = crop_image(dst_left, roi)
dst_right_crop = crop_image(dst_right, roi)

In [13]:
np.shape(dst_left_crop) == np.shape(dst_right_crop)

True

In [14]:
cv2.imwrite('dst_left_crop.png', dst_left_crop)
cv2.imwrite('dst_right_crop.png', dst_right_crop)

True

6. Для карты диспартности нам понадобятся некоторые вспомогательные функции. Одна из них просто сохраняет файл ply в удобном виде, а другая - понижает качество изображения для ускрения

In [15]:
def create_output(vertices, colors, filename):
    colors = colors.reshape(-1,3)
    vertices = np.hstack([vertices.reshape(-1,3),colors])

    ply_header = '''ply
        format ascii 1.0
        element vertex %(vert_num)d
        property float x
        property float y
        property float z
        property uchar red
        property uchar green
        property uchar blue
        end_header
        '''
    with open(filename, 'w') as f:
        f.write(ply_header %dict(vert_num=len(vertices)))
        np.savetxt(f,vertices,'%f %f %f %d %d %d')
def downsample_image(image, reduce_factor):
    for i in range(0,reduce_factor):
        #Check if image is color or grayscale
        if len(image.shape) > 2:
            row,col = image.shape[:2]
        else:
            row,col = image.shape

        image = cv2.pyrDown(image, dstsize= (col//2, row // 2))
    return image

7. Теперь используя алгоритм SGBM мы находим карту диспартности

In [16]:



win_size = 5
min_disp = -1
max_disp = 63 #min_disp * 9
num_disp = max_disp - min_disp # Needs to be divisible by 16
stereo = cv2.StereoSGBM_create(minDisparity= 0,
    numDisparities = 16,
    blockSize = 3,
    uniquenessRatio = 0,
    speckleWindowSize = 0,
    speckleRange = 0,
    disp12MaxDiff = 0,
    P1 = 0,#8*3*win_size**2,
    P2 = 0) #32*3*win_size**2)

#Compute disparity map
#print ("\nComputing the disparity  map...")
dst_left_crop_d = downsample_image(dst_left_crop, 3)
dst_right_crop_d = downsample_image(dst_right_crop, 3)
#plt.imshow(dst_left)
cv2.imwrite('dst_left_crop_d.png', dst_left_crop_d)
#plt.imshow(dst_right)
cv2.imwrite('dst_right_crop_d.png', dst_right_crop_d)

disparity_map = stereo.compute(dst_left_crop_d, dst_right_crop_d)
cv2.imwrite('disparity_map.png', disparity_map)

True

8. Восстанавливаем 3D карту при помощи reprojectImageTo3D()

In [20]:


focal_length = 8
Q2 = np.float32([[1,0,0,0],
                [0,-1,0,0],
                [0,0,focal_length*0.05,0], #Focal length multiplication obtained experimentally. 
                [0,0,0,1]])

points_3D = cv2.reprojectImageTo3D(disparity_map, Q2)
#Get color points
colors = cv2.cvtColor(dst_left_crop_d, cv2.COLOR_BGR2RGB)

#Get rid of points with value 0 (i.e no depth)
mask_map = disparity_map > disparity_map.min() #and disparity_map < disparity_map.max()

#Mask colors and points. 
output_points = points_3D[mask_map]
output_colors = colors[mask_map]

#Define name for output file
output_file = 'cotik.ply'


ox_p = np.array([[i, 0, 0] for i in range(50)])
oy_p = np.array([[0, i, 0] for i in range(50)])
oz_p = np.array([[0, 0, i] for i in range(50)])
ox_c = np.array([[255, 0, 0] for i in range(50)])
oy_c = np.array([[0, 255, 0] for i in range(50)])
oz_c = np.array([[0, 0, 255] for i in range(50)])
axis_p = np.concatenate((ox_p, oy_p, oz_p))
axis_c = np.concatenate((ox_c, oy_c, oz_c))
output_points = np.concatenate((output_points, axis_p))
output_colors = np.concatenate((output_colors, axis_c))



#Generate point cloud 
print ("\n Creating the output file... \n")
create_output(output_points, output_colors, output_file)


 Creating the output file... 

